In [65]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Bbox detection of attachments

This notebook demonstrates how to detect bounding boxes of telephone attachements images from Imagery Insights Utility pole dataset using the Gemini 2.5 Flash model.

## Install Required Libraries

In [ ]:
%pip install --upgrade google-cloud-bigquery google-cloud-aiplatform

## Configuration

**Important**: Replace the placeholder values below with your actual GCP Project ID and Region.

In [ ]:
PROJECT_ID = ''  # @param {type:"string"}
REGION = ''      # @param {type:"string"}

# BigQuery Configuration
BIGQUERY_DATASET_ID = '' # @param {type:"string"}
BIGQUERY_TABLE_ID = 'latest_observations' # @param {type:"string"}
QUERY_LIMIT = 10 # @param {type:"integer"}
ASSET_TYPE = "ASSET_CLASS_UTILITY_POLE" # @param {type:"string"}
MODEL = "gemini-2.5-flash" # @param {type:"string"}

## Imports the genAI libaries

In [68]:
import vertexai
from google.cloud import bigquery
from google import genai
from google.genai.types import (
    GenerateContentConfig,
    HarmBlockThreshold,
    HarmCategory,
    HttpOptions,
    Part,
    SafetySetting,
)
from pydantic import BaseModel



In [69]:
# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=REGION)

## Create vertex client

In [70]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=REGION)

## Fetch Image URIs from BigQuery

Next, we'll query a BigQuery table to get the GCS URIs of the images we want to classify.

In [71]:
PROMPT = """You will be provided with a photo of a utility pole:
{photo_of_utility_pole}

Output the positions of the various utility pole attachments and other objects in the photo.
Clearly label them.
DO NOT try to detect anything that is not a telephone utility
"""

# Helper class to represent a bounding box

In [72]:

class BoundingBox(BaseModel):
    """
    Represents a bounding box with its 2D coordinates and associated label.

    Attributes:
        box_2d (list[int]): A list of integers representing the 2D coordinates of the bounding box,
                            typically in the format [y_min, x_min, y_max, x_max].
        label (str): A string representing the label or class associated with the object within the bounding box.
    """

    box_2d: list[int]
    label: str

In [73]:
config = GenerateContentConfig(
    system_instruction="""
    Return bounding boxes as an array with labels.
    Detect only objects that are related to telecommunications.
    Never return masks. Limit to 25 objects.
    If an object is present multiple times, give each object a unique label
    according to its distinct characteristics (colors, size, position, etc..).
    """,
    temperature=0.5,
    safety_settings=[
        SafetySetting(
            category=HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
            threshold=HarmBlockThreshold.BLOCK_ONLY_HIGH,
        ),
    ],
    response_mime_type="application/json",
    response_schema=list[BoundingBox],
)

## Define Image Classification Function

This function takes a GCS URI and a prompt, then uses the Gemini 1.5 Flash model to generate a description of the image.

In [74]:
def classify_image_with_gemini(gcs_uri: str, prompt: str) -> str:
    """
    Classifies an image using the Gemini 2.5 Flash model by directly passing its GCS URI.
    """
    try:
          #contents = [
          #PROMPT,
          #Part(file_data={'file_uri': gcs_uri, 'mime_type': 'image/jpeg'})]

          #response = client.models.generate_content(model=MODEL, contents=contents)

          response = client.models.generate_content(
            model=MODEL,
            contents=[
            Part.from_uri(
              file_uri=gcs_uri,
              mime_type="image/jpeg",
            ),
            PROMPT,
          ],
    config=config,
)
          return response.text
    except Exception as e:
        print(f"Error classifying image from URI {gcs_uri}: {e}")
        return "Classification failed."

#Get the image URIs from BigQuery

In [ ]:
from google.cloud import bigquery

BIGQUERY_SQL_QUERY = f"""
SELECT
  *
FROM
  `{PROJECT_ID}.{BIGQUERY_DATASET_ID}.{BIGQUERY_TABLE_ID}`
  WHERE asset_type = '{ASSET_TYPE}'

LIMIT {QUERY_LIMIT};
"""

# Execute BigQuery Query
try:
    bigquery_client = bigquery.Client(project=PROJECT_ID)
    query_job = bigquery_client.query(BIGQUERY_SQL_QUERY)
    query_response_data = [dict(row) for row in query_job]
    gcs_uris = [item.get("gcs_uri") for item in query_response_data if item.get("gcs_uri")]

    print(f"Successfully fetched {len(gcs_uris)} GCS URIs:")
    for uri in gcs_uris:
        print(uri)
except Exception as e:
    print(f"An error occurred while querying BigQuery: {e}")

## Classify Images

Finally, we loop through the GCS URIs we fetched and pass them to our classification function along with a prompt.

In [76]:
if 'gcs_uris' in locals() and gcs_uris:
    for uri in gcs_uris:
        print(f"--- Classifying {uri} ---")
        classification = classify_image_with_gemini(uri, PROMPT)
        print(f"Result: {classification}\n")
else:
    print("No GCS URIs were found to classify.")

--- Classifying gs://geoai_published_ba3433e0-d709-4622-9577-66ab23587729__us/e8761cd3-c872-43cd-90aa-3ee99378090f/v0/o1:566314da74215735b48631b99f3b8429:00250082.jpg ---
Result: [
  {
    "box_2d": [359, 439, 442, 804],
    "label": "antenna"
  },
  {
    "box_2d": [657, 362, 706, 401],
    "label": "telecommunication equipment box"
  }
]

--- Classifying gs://geoai_published_ba3433e0-d709-4622-9577-66ab23587729__us/e8761cd3-c872-43cd-90aa-3ee99378090f/v0/o1:01cdba18fbed09262fba4f92e67ddb00:00250082.jpg ---
Result: [
  {
    "box_2d": [404, 574, 599, 597],
    "label": "telecommunication mast 1"
  },
  {
    "box_2d": [480, 693, 599, 697],
    "label": "telecommunication mast 2"
  }
]

--- Classifying gs://geoai_published_ba3433e0-d709-4622-9577-66ab23587729__us/e8761cd3-c872-43cd-90aa-3ee99378090f/v0/o1:b83b2016a26a802b1ccb71c27753d65c:00250082.jpg ---
Result: [
  {
    "box_2d": [240, 690, 840, 720],
    "label": "telecommunication cable on right pole"
  },
  {
    "box_2d": [240, 4